# MESSI Cluster Analysis

Robert Wright | r.wright@fu-berlin.de

In [2]:
import pandas as pd

### DATA IMPORT AND TIDYING

In [43]:
# import temperature data and parse measurement time as datetime object
meas = pd.read_csv("../data/data.csv", parse_dates=["time_measured"])
meas

,time_measured,2,8,9,13,15,17,20,21,22,...,76,79,80,85,88,91,400,433,427,420
0,2022-08-01 00:00:00,21.690,21.930,NaN,NaN,NaN,22.470,NaN,20.770,21.460,...,NaN,NaN,NaN,20.930,21.150,NaN,20.4,21.3,21.6,21.0
1,2022-08-01 00:10:00,21.530,21.865,NaN,NaN,NaN,22.475,NaN,20.510,21.260,...,NaN,NaN,NaN,20.740,21.000,NaN,20.4,21.2,21.5,20.9
2,2022-08-01 00:20:00,21.260,21.720,NaN,NaN,NaN,22.395,NaN,19.290,21.110,...,NaN,NaN,NaN,20.550,20.900,NaN,20.3,20.9,21.2,20.9
3,2022-08-01 00:30:00,21.075,21.485,NaN,NaN,NaN,22.295,NaN,18.700,20.960,...,NaN,NaN,NaN,20.435,20.625,NaN,20.2,20.7,21.2,20.7
4,2022-08-01 00:40:00,21.030,21.325,NaN,NaN,NaN,22.030,NaN,18.490,20.990,...,NaN,NaN,NaN,20.405,20.565,NaN,19.9,20.7,21.0,20.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,2022-08-31 23:20:00,16.455,15.940,NaN,NaN,NaN,16.040,16.265,12.950,16.305,...,NaN,NaN,16.425,NaN,NaN,NaN,14.1,NaN,15.4,15.0
4461,2022-08-31 23:30:00,16.280,15.865,NaN,NaN,NaN,16.025,16.060,13.430,16.450,...,NaN,NaN,16.455,NaN,NaN,NaN,14.0,NaN,15.4,14.9
4462,2022-08-31 23:40:00,16.060,15.810,NaN,NaN,NaN,16.025,15.930,13.535,16.130,...,NaN,NaN,16.230,NaN,16.030,NaN,14.0,NaN,15.3,14.8
4463,2022-08-31 23:50:00,NaN,15.780,NaN,NaN,NaN,15.925,15.930,13.780,15.990,...,NaN,NaN,16.260,NaN,15.865,NaN,14.0,NaN,15.3,14.8


In [60]:
# import location of dwd stations
dwd_loc = pd.read_csv("../data/stationen_berlin_brandenburg.txt", delim_whitespace=True)

dwd_loc = (dwd_loc
    .drop(["von_datum", "bis_datum", "Bundesland"], axis="columns")
    .rename(columns = {"Stationshoehe": "alt", "geoBreite": "lat", "geoLaenge": "lon", "Stationsname": "station_name"})
    .set_index("Stations_id")
)

dwd_loc

,alt,lat,lon,station_name
Stations_id,,,,
399,36,52.5198,13.4057,Berlin-Alexanderplatz
400,60,52.6310,13.5021,Berlin-Buch
410,33,52.4040,13.7309,Berlin-Kaniswall
420,61,52.5447,13.5598,Berlin-Marzahn
427,46,52.3807,13.5306,Berlin-Brandenburg
430,36,52.5644,13.3088,Berlin-Tegel
433,48,52.4675,13.4021,Berlin-Tempelhof


In [40]:
# import location & altitude of messis
messi_loc = pd.read_csv("../data/messi_standorte.csv")
# set id as index for joining
messi_loc.set_index("gauge_id", inplace=True)
messi_loc

,lon,lat,alt
gauge_id,,,
2,13.33395,52.48429,42
8,13.47643,52.44803,34
9,13.31014,52.45784,72
13,13.55526,52.55220,52
17,13.52497,52.45489,34
20,13.34865,52.44442,35
21,13.29767,52.60324,39
22,13.35322,52.46731,43
23,13.40305,52.44976,39


In [35]:
meas

,time_measured,2,8,9,13,15,17,20,21,22,...,76,79,80,85,88,91,400,433,427,420
0,2022-08-01 00:00:00,21.690,21.930,NaN,NaN,NaN,22.470,NaN,20.770,21.460,...,NaN,NaN,NaN,20.930,21.150,NaN,20.4,21.3,21.6,21.0
1,2022-08-01 00:10:00,21.530,21.865,NaN,NaN,NaN,22.475,NaN,20.510,21.260,...,NaN,NaN,NaN,20.740,21.000,NaN,20.4,21.2,21.5,20.9
2,2022-08-01 00:20:00,21.260,21.720,NaN,NaN,NaN,22.395,NaN,19.290,21.110,...,NaN,NaN,NaN,20.550,20.900,NaN,20.3,20.9,21.2,20.9
3,2022-08-01 00:30:00,21.075,21.485,NaN,NaN,NaN,22.295,NaN,18.700,20.960,...,NaN,NaN,NaN,20.435,20.625,NaN,20.2,20.7,21.2,20.7
4,2022-08-01 00:40:00,21.030,21.325,NaN,NaN,NaN,22.030,NaN,18.490,20.990,...,NaN,NaN,NaN,20.405,20.565,NaN,19.9,20.7,21.0,20.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4460,2022-08-31 23:20:00,16.455,15.940,NaN,NaN,NaN,16.040,16.265,12.950,16.305,...,NaN,NaN,16.425,NaN,NaN,NaN,14.1,NaN,15.4,15.0
4461,2022-08-31 23:30:00,16.280,15.865,NaN,NaN,NaN,16.025,16.060,13.430,16.450,...,NaN,NaN,16.455,NaN,NaN,NaN,14.0,NaN,15.4,14.9
4462,2022-08-31 23:40:00,16.060,15.810,NaN,NaN,NaN,16.025,15.930,13.535,16.130,...,NaN,NaN,16.230,NaN,16.030,NaN,14.0,NaN,15.3,14.8
4463,2022-08-31 23:50:00,NaN,15.780,NaN,NaN,NaN,15.925,15.930,13.780,15.990,...,NaN,NaN,16.260,NaN,15.865,NaN,14.0,NaN,15.3,14.8


In [52]:
data = meas.melt(id_vars="time_measured", var_name="station_id", value_name="tmp")
data['station_id'] = data["station_id"].astype(int)
data.set_index("station_id", inplace=True)
data


,time_measured,tmp
station_id,,
2,2022-08-01 00:00:00,21.690
2,2022-08-01 00:10:00,21.530
2,2022-08-01 00:20:00,21.260
2,2022-08-01 00:30:00,21.075
2,2022-08-01 00:40:00,21.030
...,...,...
420,2022-08-31 23:20:00,15.000
420,2022-08-31 23:30:00,14.900
420,2022-08-31 23:40:00,14.800


In [64]:
test = data.join(other=[messi_loc, dwd_loc])
test

,time_measured,tmp,lon_x,lat_x,alt_x,alt_y,lat_y,lon_y,station_name
2,2022-08-01 00:00:00,21.690,13.33395,52.48429,42.0,NaN,NaN,NaN,NaN
2,2022-08-01 00:10:00,21.530,13.33395,52.48429,42.0,NaN,NaN,NaN,NaN
2,2022-08-01 00:20:00,21.260,13.33395,52.48429,42.0,NaN,NaN,NaN,NaN
2,2022-08-01 00:30:00,21.075,13.33395,52.48429,42.0,NaN,NaN,NaN,NaN
2,2022-08-01 00:40:00,21.030,13.33395,52.48429,42.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
433,2022-08-31 23:20:00,NaN,NaN,NaN,NaN,48.0,52.4675,13.4021,Berlin-Tempelhof
433,2022-08-31 23:30:00,NaN,NaN,NaN,NaN,48.0,52.4675,13.4021,Berlin-Tempelhof
433,2022-08-31 23:40:00,NaN,NaN,NaN,NaN,48.0,52.4675,13.4021,Berlin-Tempelhof
433,2022-08-31 23:50:00,NaN,NaN,NaN,NaN,48.0,52.4675,13.4021,Berlin-Tempelhof
